### Information Retrieval Pipeline
In this notebook, we prototype the IR product that will <b>recieve an input string and return the most relevant offers</b> to the user with respect to that code. These are the steps taken to achieve this:
- Load libraries and define functions.
- Given an input text, <b>get the category</b> classified by our TypeDetector service which we consume through an API.
- Depending on the response (0 = Retail, 1 = Brand, 2 = Category) we perform a specific treatment.
- If the <b>type of query is 0 or 1</b>, we search for a match with our unique retailers or brands by performing <b>TF-IDF</b> and <b>Cosine Similarity</b>.
- Also, we compute a '<b>fuzzy similarity</b> score' in case there are typeos, which can be helpful if the cosine similarity doesn't bring relevant results.
- Next, we fetch offers that come from our matched retailers or brands, only if the cosine similarity is > 0.9 or the fuzzy similarity is > 0.75
- If the <b>type of query is 2</b>, then the similarity of the term is compared to offers directly, and synonyms of the terms are computed and also compared to the offers to contemplate a wider range of relevant results.
- Finally, the relevant offers are returned in a json list along with a log of activity.

<b>*IMPORTANT NOTE</b>: the similarity score is only returned if the type of query is 'category' as in the other cases the only match is with respect to the retailer or brand and that similarity is returned in the log.

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from fuzzywuzzy import fuzz, process
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet
from textblob import TextBlob
import pandas as pd
import numpy as np
import requests
import json
import nltk
import re

nltk.download('wordnet')
nltk.download('punkt')

/usr/local/lib/python3.8/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
# Define some functions to use in the process

def isNaN(num):
    return num != num

def getSynonyms(word):
    synonyms = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.append(lemma.name().replace('_', ' '))
    # Remove duplicates
    synonyms = list(set(synonyms))
    return synonyms

def listToJson(string_list):
    json_list = []
    for item in string_list:
        json_object = {
            'concept': item,
            'cosine_similarity': -1,
            'fuzzy_score': -1
        }
        json_list.append(json_object)
    return json_list

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stemmer = PorterStemmer()
    return [stemmer.stem(t) for t in tokens]

def getTopSortedSimilarityMatches(input_text, target_list):
    '''
    Returns the top sorted matches of the list with respect to the input text (TF-IDF -> cosine similarity) if similarity > 0,
    If no matches are found, it triew using 'fuzzy' matches to compensate for misspelling.
    '''
    documents = [input_text] + target_list
    tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenize, ngram_range=(1, 3))

    tfidf_matrix = tfidf_vectorizer.fit_transform(documents)
    cosine_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()

    sorted_index = np.argsort(-cosine_similarities)
    sorted_retailers = [target_list[i] for i in sorted_index]
    sorted_similarities = [cosine_similarities[i] for i in sorted_index]

    lst_resp = []

    for i, retailer in enumerate(sorted_retailers):
        if sorted_similarities[i] > 0:
            lst_resp.append({'concept': retailer, 'cosine_similarity': sorted_similarities[i], 'fuzzy_score' : -1})
    
    # If the list is empty, try using fuzzy matching as a fallback
    if not lst_resp:
        highest = process.extractOne(input_text, target_list)
        if highest:
            lst_resp.append({'concept': highest[0], 'cosine_similarity':-1, 'fuzzy_score': highest[1]/100.0})
    
    return lst_resp

def getCategory(text_input):
    '''
    This function calls our other 'QueryTypeDetector' service and returns the category and confidence.
    '''
    url = "http://20.51.242.109:5000/predict"
    data = {"text": text_input}
    headers = {"Content-Type": "application/json"}
    
    response = requests.post(url, data=json.dumps(data), headers=headers)
    #print("Status Code:", response.status_code)
    return response.json()

In [12]:
# Let's define a sample 'input' to test the pipeline and store in the log.

input_text = 'sams club membership'
log_list = [f'Input text recieved was "{input_text}"']

In [13]:
# Now, let's call our TypeDetector service

category_json = getCategory(input_text)
deducted_category = category_json['prediction']
log_list.append(f'Input categorized by QueryTypeDetection : {category_json["prediction"]} with confidence: {category_json["confidence"]} (0=retail, 1=brand, 2=category)')
deducted_category

0

In [17]:
# Let's get unique retailers, brands and categories

df_offer_retailer = pd.read_csv('data/offer_retailer.csv')
df_cat = pd.read_csv('data/categories.csv')
df_brand_category = pd.read_csv('data/brand_category.csv')

unique_retailers = list(set([r['RETAILER'] for i, r in df_offer_retailer.iterrows() if not isNaN(r['RETAILER'])]))
unique_brands = list(set([r['BRAND'] for i, r in df_offer_retailer.iterrows() if (r['BRAND'] != r['RETAILER']) & (not isNaN(r['BRAND']))]))
unique_categories = list(set([r['PRODUCT_CATEGORY'] for i, r in df_cat.iterrows() if not isNaN(r['PRODUCT_CATEGORY'])]))
unique_categories += list(set([r['IS_CHILD_CATEGORY_TO'] for i, r in df_cat.iterrows() if not isNaN(r['IS_CHILD_CATEGORY_TO'])]))
unique_offers = list(set([r['OFFER'] for i, r in df_offer_retailer.iterrows() if not isNaN(r['OFFER'])]))

print(f'Number of unique retailers: {len(unique_retailers)}')
print(f'Number of unique brands: {len(unique_brands)}')
print(f'Number of unique categories: {len(unique_categories)}')
print(f'Number of unique offers: {len(unique_offers)}')

Number of unique retailers: 61
Number of unique brands: 121
Number of unique categories: 141
Number of unique offers: 376


In [18]:
# Let's extract the 'similarity_list' to use as anchor to fetch the offers, each case requires a special treatment.

offer_list = []

if deducted_category == 0:
    similarity_list = getTopSortedSimilarityMatches(input_text, unique_retailers)
    log_list.append(f'Similarity matches with retailers: {similarity_list}')
    if len(similarity_list) > 0 and (similarity_list[0]['cosine_similarity'] > 0.9 or similarity_list[0]['fuzzy_score'] > 0.75):
        offer_list = getTopSortedSimilarityMatches(input_text, list(df_offer_retailer[df_offer_retailer['RETAILER'] == similarity_list[0]['concept']]['OFFER']))
elif deducted_category == 1:
    similarity_list = getTopSortedSimilarityMatches(input_text, unique_brands)
    log_list.append(f'Similarity matches with brands: {similarity_list}')
    if len(similarity_list) > 0 and (similarity_list[0]['cosine_similarity'] > 0.9 or similarity_list[0]['fuzzy_score'] > 0.75):
        offer_list = getTopSortedSimilarityMatches(input_text, list(df_offer_retailer[df_offer_retailer['BRAND'] == similarity_list[0]['concept']]['OFFER']))
else:
    lst_syn = [getTopSortedSimilarityMatches(word, unique_categories) for word in [input_text] + getSynonyms(input_text)]
    flat_list = [elem for sublist in lst_syn for elem in sublist]
    # Remove duplicates
    max_similarity_dict = {}
    for item in flat_list:
        concept = item['concept']
        similarity = item['cosine_similarity']
        if concept not in max_similarity_dict or similarity > max_similarity_dict[concept]['cosine_similarity']:
            max_similarity_dict[concept] = item
    
    # Convert the dictionary values to a list to get the final, deduplicated list
    similarity_list = list(max_similarity_dict.values())
    similarity_list = [item for item in similarity_list if item['cosine_similarity'] > 0 or item['fuzzy_score'] > 0.85]
    similarity_list = sorted(similarity_list, key=lambda x: x['cosine_similarity'], reverse=True)
    log_list.append(f'Similarity matches with categories: {similarity_list}')

    # Get the offer list comparing directly to offers
    offer_list = [getTopSortedSimilarityMatches(word, unique_offers) for word in [input_text] + getSynonyms(input_text)]
    offer_list = [elem for sublist in offer_list for elem in sublist]
    offer_list = sorted(offer_list, key=lambda x: x['cosine_similarity'], reverse=True)
    '''
    # LINK MATCHED CATEGORIES TO BRANDS AND THEN LINK BRANDS TO OFFERS (TO DO IN LATER STAGES)
    unique_target_brands = []
    for item in similarity_list:
        filtered_df = df_brand_category[df_brand_category['BRAND_BELONGS_TO_CATEGORY'] == item['concept']]
        unique_target_brands += list(filtered_df['BRAND'].unique())
    unique_target_brands = list(set(unique_target_brands))
    '''

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [19]:
# Finally, we wrap the response in a json to return as a service.
response = {'offer_list':offer_list, 'LOG_LIST':log_list}
response

{'offer_list': [{'concept': 'Spend $50 on a Full-Priced new Club Membership',
   'cosine_similarity': 0.17167737713678186,
   'fuzzy_score': -1},
  {'concept': "George's Farmers Market Chicken Wings, at Sam's Club",
   'cosine_similarity': 0.07507578805263351,
   'fuzzy_score': -1},
  {'concept': "Tyson Products, select varieties, spend $20 at Sam's Club",
   'cosine_similarity': 0.06966444274341704,
   'fuzzy_score': -1},
  {'concept': 'Spend $110 on a Full-Priced new Plus Membership and receive an ADDITIONAL 10,000 points',
   'cosine_similarity': 0.037681749600967825,
   'fuzzy_score': -1}],
 'LOG_LIST': ['Input text recieved was "sams club membership"',
  'Input categorized by QueryTypeDetection : 0 with confidence: 0.9963080883026123 (0=retail, 1=brand, 2=category)',
  "Similarity matches with retailers: [{'concept': 'SAMS CLUB', 'cosine_similarity': 0.6725889529793223, 'fuzzy_score': -1}]",
  "Similarity matches with retailers: [{'concept': 'SAMS CLUB', 'cosine_similarity': 0.672